# Visual Odometry
## Done by: Projit and Abhinav 
(Abhinav wrote the function implementing 8-point algorithm and Projit worked on the RANSAC loop and scaling. Both worked on the updates in the main loop.)

We calculate the trajectory of the vehicle, using a basic monocular visual odometry algorithm. 

Importing libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy.linalg as linalg
import random

A bunch of utility functions to help us through the way:

In [2]:
# Utility Functions

def display_image(img):
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img2)
    plt.show()

def draw_line(img, A, B):
    cv2.line(img, A, B, (0, 255, 0), thickness=3, lineType=8)

def add_column(original, newcol):
    return np.concatenate([original, newcol], axis=1)

def add_row(original, newrow): # didn't use
    return np.concatenate([original, newrow], axis=0)

def draw_point(image, point, color):
    if color == "green":
        cv2.circle(image, point, 10, (0, 255, 0), thickness=3, lineType=8)
    elif color == "red":
        cv2.circle(image, point, 8, (0, 0, 255), thickness=3, lineType=8)


## Finding Matches
Given the keypoints and descriptors, we want to find corresponding features between two frames. So we use Lowe's ratio test to find the good matches. Matches are done using a brute force KNN. 0.15 is the hyperparameter we choose to ensure that we don't have matches in the same area of the image.  

In [3]:
# Finding matches
bf = cv2.BFMatcher()
def find_matches(des1, des2, kp1, kp2):
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for m in matches:
        if m[0].distance < 0.15*m[1].distance:
            good.append(m)
    if len(good) > 10:
        matches = np.asarray(good)
    else:
        matches = np.asarray(matches)
    if len(matches[:,0]) >= 8:
        src = np.float32([ kp1[m.queryIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
        dst = np.float32([ kp2[m.trainIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
    src_new = []
    dst_new = []
    for i in range(len(src)):
        src_new.append(src[i][0])
        dst_new.append(dst[i][0])
    return src_new, dst_new


## Computing the Fundamental Matrix
We use the 8-point algorithm to compute the fundamental matrix. We take the 8 points in A and then find the eigenvectors and select based on the eigenvalue. Here is a function that does this: 

In [4]:
# Check somewhere here that at least 
def compute_fundamental(src, dst):
    A = np.array([
    [src[0][0]*dst[0][0], src[0][0]*dst[0][1], src[0][0], src[0][1]*dst[0][0], src[0][1]*dst[0][1], src[0][1], dst[0][0], dst[0][1], 1],
    [src[1][0]*dst[1][0], src[1][0]*dst[1][1], src[1][0], src[1][1]*dst[1][0], src[1][1]*dst[1][1], src[1][1], dst[1][0], dst[1][1], 1],
    [src[2][0]*dst[2][0], src[2][0]*dst[2][1], src[2][0], src[2][1]*dst[2][0], src[2][1]*dst[2][1], src[2][1], dst[2][0], dst[2][1], 1],
    [src[3][0]*dst[3][0], src[3][0]*dst[3][1], src[3][0], src[3][1]*dst[3][0], src[3][1]*dst[3][1], src[3][1], dst[3][0], dst[3][1], 1],
    [src[4][0]*dst[4][0], src[4][0]*dst[4][1], src[4][0], src[4][1]*dst[4][0], src[4][1]*dst[4][1], src[4][1], dst[4][0], dst[4][1], 1],
    [src[5][0]*dst[5][0], src[5][0]*dst[5][1], src[5][0], src[5][1]*dst[5][0], src[5][1]*dst[5][1], src[5][1], dst[5][0], dst[5][1], 1],
    [src[6][0]*dst[6][0], src[6][0]*dst[6][1], src[6][0], src[6][1]*dst[6][0], src[6][1]*dst[6][1], src[6][1], dst[6][0], dst[6][1], 1],
    [src[7][0]*dst[7][0], src[7][0]*dst[7][1], src[7][0], src[7][1]*dst[7][0], src[7][1]*dst[7][1], src[7][1], dst[7][0], dst[7][1], 1],
    ])
    B = np.matmul(A.transpose(), A)
    eigenValues, eigenVectors = linalg.eig(B)
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    temp = eigenVectors[1]
    #temp = eigenVectors[-1]
    F = np.array([
        [temp[0], temp[1], temp[2]],
        [temp[3], temp[4], temp[5]],
        [temp[6], temp[7], temp[8]]
    ])
    return np.real(F)

## RANSAC
We have a bunch of matches with us. Now we want to calculate a fundamental matrix and evaluate how accurate it is on these points. So we randomly select 8 points, calculate F, and see how close x' F x gets to zero. That is, our metric is to minimize x' F x = 0. Closer to 0 would imply a better F matrix

In [5]:
# RANSAC loop
def RANSAC_LOOP(src, dst):
    RANSAC_TIMES = 800
    num_vals = len(src)
    minval = 9999999
    F_hold = None
    for i in range(RANSAC_TIMES):
        # Choose random 8.
        rand_choice = [random.randint(0, num_vals-1) for x in range(8)]
        src_input = [src[x] for x in rand_choice]
        dst_input = [dst[x] for x in rand_choice]
        F_temp = compute_fundamental(src_input, dst_input)
        tot = 0
        for j in range(num_vals):
            vec1 = np.transpose(np.array([dst[j][0], dst[j][1], 1]))
            vec2 = np.array([src[j][0], src[j][1], 1])
            temp2 = np.matmul(vec1, F_temp)
            tot += np.matmul(temp2, vec2)
        if tot < 0:
            tot = -1 * tot
        if tot < minval:
            F_hold = F_temp
            minval = tot
    print(minval)
    return F_hold

## Loading the dataset
We load the KITTI datset and we extract SIFT kepypoints and descriptions from all the images. 

In [6]:
# Setup, run this only once.
K = np.array([[7.215377000000e+02,0.000000000000e+00,6.095593000000e+02],
              [0.000000000000e+00,7.215377000000e+02,1.728540000000e+02],
              [0.000000000000e+00,0.000000000000e+00,1.000000000000e+00]])

# Loading all the images.
dir_im = os.listdir('./images')
dir_im.sort()
images = [cv2.imread(os.path.join('./images', x)) for x in dir_im]

# Extracting all the sift features from each of the images.
features = []
sift = cv2.xfeatures2d.SIFT_create()

for im in images:
    kp1, des1 = sift.detectAndCompute(cv2.cvtColor(im, cv2.COLOR_BGR2GRAY), None)
    features.append({"kp": kp1, "des": des1})
    

FileNotFoundError: [Errno 2] No such file or directory: './images'

## Scaling
This function computes the norm between the ground truth and our predicted vector. We use this as a sacaling factor. 

In [7]:
def get_absolute_scale(pose_c, index):
    pose = pose_c[index - 1].strip().split()
    x_prev = float(pose[3])
    y_prev = float(pose[7])
    z_prev = float(pose[11])
    pose = pose_c[index].strip().split()
    x = float(pose[3])
    y = float(pose[7])
    z = float(pose[11])

    true_vect = np.array([[x], [y], [z]])
    prev_vect = np.array([[x_prev], [y_prev], [z_prev]])

    return np.linalg.norm(true_vect - prev_vect)

## The main loop
- First, for every pair of image frames, we find the matches.
- Then we run RANSAC to estimate the fundamental matrix between the two images. We get the best fundamental matrix from this. 
- We then calculate the eseential matrix from the fundamental one by K<sup>T</sup>FK
- Then, we recover R and t from `cv2.recoverPose`
- Then we compose the previous rotations and translations to create the current position by:
`t_old = t_old + np.linalg.inv(R).dot(t) * abs_scale `
- We then update R by `R_old = R.dot(R_old)`
- Concatenating R and t and we print it onto the results file. 


In [44]:
def main():
    strf = ''

    R_old = np.eye(3); # np.zeros((3, 3))
    t_old = np.zeros((3, 1))
    P = add_column(R_old, t_old)
    P = add_row(P, np.array([[0, 0, 0, 1]]))
    with open('./ground-truth.txt') as f:
        pose_c = f.readlines()

    #for i in range(1, 100:
    for i in range(1, len(images)):
        src, dst = find_matches(features[i-1]['des'], features[i]['des'], features[i-1]['kp'], features[i]['kp'])
        src = np.array(src)
        dst = np.array(dst)
        F = RANSAC_LOOP(src, dst)
        E = np.matmul(np.matmul(K.T, F), K)
        print('E:', E)
        print('F:', F)
        _, R, t, mask = cv2.recoverPose(E, src, dst)
        
        if i <= 2:
            R_old = R
            abs_scale = get_absolute_scale(pose_c, i)
            t_old = t * abs_scale

        else:
            abs_scale = get_absolute_scale(pose_c, i)
            t_old = t_old + np.linalg.inv(R).dot(t) * abs_scale 
            R_old = R.dot(R_old)
            
        print("abs scale: %2.4f | progress: %d | len(src): %d " %(abs_scale, i, len(src)))
        # print(P.shape)
        for x in add_column(R_old, t_old):
            for y in x:
                strf += str("{:.6e}".format(float(y))) + ' '
        strf = strf[:-1]
        strf += '\n'
    f = open('results.txt', 'w')
    f.write(strf)

In [47]:
main()

351.01970400115533
En: [[-7.39538648e-06 -9.97665727e-04  2.61985817e-01]
 [ 9.91938811e-04  4.47658086e-06 -6.56781880e-01]
 [-2.56859951e-01  6.58803446e-01  1.25851471e-05]]
E: [[-109619.49438706  340631.70187381  -10893.05046957]
 [ 132357.10036733  344927.0439312   194437.57751987]
 [ -60898.27600497  370400.309089     37378.63913497]]
Fn: [[-5.10402448e-07 -5.40860314e-04  9.42532131e-02]
 [ 5.40653141e-04  2.94106363e-07 -3.31403291e-01]
 [-9.54304676e-02  3.30979563e-01  1.00000000e+00]]
F: [[-2.10557027e-01  6.54285068e-01  1.54208451e-01]
 [ 2.54231400e-01  6.62535557e-01 -1.43641667e-02]
 [ 1.39456823e-03  1.04894008e-03  4.45591674e-04]]
abs scale: 0.9619 | progress: 1 | len(src): 68 
6054.2386650859335
En: [[ 1.20796151e-06 -1.13171313e-03  1.48338590e-01]
 [ 1.12804906e-03  8.37885852e-06 -6.91371381e-01]
 [-1.48810862e-01  6.91269882e-01  8.04311329e-06]]
E: [[-113088.61565502  332219.71395357  -16116.47302587]
 [ 267774.21549556 -248413.70708656  166693.55532304]
 [ -31

29954.03683105283
En: [[ 2.10476280e-07 -1.09945073e-03  1.92201774e-01]
 [ 1.09871661e-03  4.98327574e-06 -6.80483114e-01]
 [-1.92431711e-01  6.80418129e-01  4.45097297e-06]]
E: [[-123838.16658824  326034.01864286  -26689.25118547]
 [ 313334.50968347 -187364.98801325  219814.19918952]
 [ -29554.71571113  230550.25054589   30113.19889628]]
Fn: [[ 9.25714402e-07  8.51167848e-03 -1.49026589e+00]
 [-8.50762919e-03  2.94616648e-07  5.22378154e+00]
 [ 1.49005370e+00 -5.23328286e+00  1.00000000e+00]]
F: [[-2.37868239e-01  6.26245851e-01 -2.43709136e-01]
 [ 6.01852646e-01 -3.59890501e-01 -9.49842823e-03]
 [ 1.42370325e-03  8.00917942e-04 -1.51786954e-04]]
abs scale: 0.8842 | progress: 13 | len(src): 54 
2993.472381294385
En: [[-1.68087352e-06 -1.11167862e-03  1.91597502e-01]
 [ 1.11120486e-03  2.50270418e-06 -6.80653480e-01]
 [-1.90679655e-01  6.80911180e-01  3.81879189e-06]]
E: [[-104471.50547487  343424.69010454   -5856.34290066]
 [  60660.80822644  360177.25986068  137531.11709591]
 [ -737

7400.536668865549
En: [[ 7.14993422e-07 -1.01851731e-03  1.45155288e-01]
 [ 1.02478275e-03 -3.76416016e-06 -6.92046887e-01]
 [-1.46366533e-01  6.91791730e-01 -6.62609960e-06]]
E: [[-110239.93438992  332105.44179204  -13742.41943304]
 [ 250656.59597885 -265283.77396229  148217.76454667]
 [ -33082.17956196  217012.40512246   23898.75045081]]
Fn: [[ 4.22238887e-07  3.77052011e-03 -6.62909621e-01]
 [-3.77411011e-03  3.61245648e-06  2.29807947e+00]
 [ 6.62059152e-01 -2.30016165e+00  1.00000000e+00]]
F: [[-2.11748767e-01  6.37907835e-01 -2.37507783e-01]
 [ 4.81460965e-01 -5.09556835e-01  1.92218388e-02]
 [ 1.42570413e-03  6.24318514e-05  9.56782386e-04]]
abs scale: 0.8632 | progress: 25 | len(src): 71 
61262.867807710805
En: [[ 2.68353706e-06 -1.12289213e-03  1.84236361e-01]
 [ 1.12541427e-03 -8.18688839e-07 -6.82682722e-01]
 [-1.86106585e-01  6.82175253e-01 -4.29673933e-06]]
E: [[-145959.45860379  316088.37905664  -47428.87208834]
 [ 366851.72344842   51688.17913739  322305.40740645]
 [ -35

110149.2743474374
En: [[ 1.96613390e-06 -1.21285301e-03  2.61948263e-01]
 [ 1.21322784e-03  2.38848022e-06 -6.56796494e-01]
 [-2.63113892e-01  6.56330411e-01  4.74730803e-08]]
E: [[-119256.5259508   321115.46783577  -23644.69477937]
 [ 332454.37103036 -164120.64038443  241550.78474418]
 [ -21105.48938748  231963.38635009   37891.00337802]]
Fn: [[-6.76069093e-07  1.06700152e-03 -1.86842784e-01]
 [-1.06756636e-03 -1.95092688e-07  6.43765230e-01]
 [ 1.86009614e-01 -6.44825173e-01  1.00000000e+00]]
F: [[-2.29067828e-01  6.16798303e-01  2.44503730e-01]
 [ 6.38578058e-01 -3.15242779e-01  1.20110321e-02]
 [-1.05575012e-03  6.10144001e-04 -1.07010909e-04]]
abs scale: 0.7446 | progress: 37 | len(src): 73 
16505.196918801827
En: [[ 1.45729914e-06 -1.05391353e-03  1.68829963e-01]
 [ 1.06126292e-03 -4.40870686e-06 -6.86655169e-01]
 [-1.70668041e-01  6.86200630e-01 -8.61532389e-06]]
E: [[-95860.33633425 343028.19087319    978.52977554]
 [332045.80691645 -99218.98357406 256746.38994182]
 [ -1436.575

388903.14528055687
En: [[ 1.19929890e-05  2.17942268e-03 -4.99953823e-01]
 [-2.16291721e-03  9.39044718e-05  5.00041633e-01]
 [ 4.85935089e-01 -5.13675181e-01 -1.42196442e-05]]
E: [[  76765.38371912 -351803.86016121  -19927.73468484]
 [ -52566.97618617  -91177.17490213  -66250.60821292]
 [  52254.02243428 -319050.31679647  -32710.65173501]]
Fn: [[ 1.98462491e-06  4.30225873e-04 -7.66355537e-02]
 [-4.29791995e-04  2.38279046e-06  2.40951747e-01]
 [ 7.34243744e-02 -2.40602757e-01  1.00000000e+00]]
F: [[ 1.47450880e-01 -6.75744540e-01 -6.93333509e-01]
 [-1.00970601e-01 -1.75133036e-01  1.37745141e-03]
 [-6.51976725e-03 -2.26200582e-03 -1.91591421e-04]]
abs scale: 0.6501 | progress: 49 | len(src): 139 
334558.2434750606
En: [[ 6.85549548e-06  1.87642792e-03 -4.49285529e-01]
 [-1.85663962e-03  5.78332163e-05  5.46021180e-01]
 [ 4.38489431e-01 -5.54728251e-01 -8.54369389e-06]]
E: [[ 119534.95498429 -282649.25639672   33553.80009256]
 [-358449.05389966   17470.90220808 -298746.32902529]
 [  1

...

20699.59902053728
En: [[-2.92184791e-04  6.46068462e-02  7.84865699e-02]
 [-6.38967638e-02  2.00404174e-03 -6.99846228e-01]
 [ 8.02836147e-02 -6.99531552e-01 -2.07878267e-03]]
E: [[ 114425.80743854 -312032.05216529   22141.29512358]
 [-363657.0283833   -34784.44864479 -315515.06018411]
 [   9549.27781891 -271938.90420519  -56880.09517971]]
Fn: [[ 1.38417847e-07  4.48731275e-04 -8.56678373e-02]
 [-4.47486155e-04  1.33959867e-06  2.26264198e-01]
 [ 8.60876348e-02 -2.33692294e-01  1.00000000e+00]]
F: [[ 2.19788988e-01 -5.99350886e-01  3.12038462e-01]
 [-6.98512094e-01 -6.68139378e-02  5.21379920e-02]
 [ 8.08376198e-04  9.58288107e-04 -3.51577047e-04]]
abs scale: 0.3144 | progress: 746 | len(src): 131 
533.5575179964944
En: [[ 3.16480804e-06 -1.66587825e-03  2.11926651e-02]
 [ 1.64975274e-03  3.95821004e-05  7.06787204e-01]
 [-1.96319620e-02 -7.06832233e-01  3.54259799e-05]]
E: [[-106402.79746102  321449.09903108  -12681.76186809]
 [ 350175.72328954 -110077.66943522  269496.47888308]
 [  -

51605.612115333475
En: [[ 7.18705869e-07  7.97287389e-04 -1.58870142e-01]
 [-7.82895287e-04  3.52889881e-06  6.89028058e-01]
 [ 1.55400927e-01 -6.89818756e-01 -3.70396207e-06]]
E: [[-107241.20698515  331396.39850072  -11066.26743114]
 [ 371427.17987814  -18927.78592176  309293.42960704]
 [  -1617.91718561  275431.26436843   64746.31631538]]
Fn: [[ 4.35580225e-07 -1.91392533e-04  3.81082811e-02]
 [ 1.92177263e-04  9.15535785e-09 -1.02452045e-01]
 [-3.88395854e-02  9.83681471e-02  1.00000000e+00]]
F: [[-2.05988814e-01  6.36545906e-01  1.95830933e-01]
 [ 7.13436995e-01 -3.63564743e-02  6.09307517e-02]
 [-3.59063665e-04  5.34281559e-05 -1.15710283e-03]]
abs scale: 0.2689 | progress: 758 | len(src): 134 
57679.908672647485
En: [[ 5.29876460e-07 -4.48338874e-04  9.87298033e-02]
 [ 4.35309912e-04  2.05477127e-05 -7.00180143e-01]
 [-9.73440646e-02  7.00374139e-01  2.28261657e-05]]
E: [[ -94838.42021125  352810.67391643    4301.75221372]
 [ 308723.71494967 -192064.42867312  214776.0374724 ]
 [ 

1314.5203234432356
En: [[ 1.36405924e-06 -9.66374410e-04  6.13884301e-02]
 [ 9.70743981e-04  2.22838580e-06 -7.04436313e-01]
 [-6.26629425e-02  7.04324088e-01  8.74038586e-08]]
E: [[-67610.90076163 355941.43226755  28249.06482759]
 [-65349.22992241 357792.6973586   30435.64622024]
 [-72769.52336371 386415.76914914  31159.56922288]]
Fn: [[ 5.28439112e-06 -4.49026270e-02  7.74054992e+00]
 [ 4.49167034e-02 -4.13564019e-05 -2.51958663e+01]
 [-7.76367938e+00  2.52549101e+01  1.00000000e+00]]
F: [[-1.29866958e-01  6.83691984e-01  1.33914791e-01]
 [-1.25522742e-01  6.87247893e-01 -9.83485453e-02]
 [ 5.32947164e-03  4.53524429e-04 -4.96392655e-05]]
abs scale: 0.2497 | progress: 770 | len(src): 137 
925.9967431891419
En: [[ 3.15729518e-06 -1.10809290e-03  4.74042570e-02]
 [ 1.11488202e-03  6.08471808e-06 -7.05515126e-01]
 [-4.97120773e-02  7.05356280e-01  1.96957252e-06]]
E: [[-115153.28325194  333013.84891878  -17351.40661366]
 [ 200522.52163344  298596.28350633  241036.7112632 ]
 [ -49242.866

77366.23091674689
En: [[-7.37175256e-06 -4.35083151e-04  3.17652606e-01]
 [ 4.23459502e-04  1.52501988e-05  6.31740966e-01]
 [-3.14992200e-01 -6.33071654e-01  4.26668756e-06]]
E: [[-131875.48675109  262017.94558819  -48336.03048943]
 [ 364593.94773884  -50978.17279295  295744.58982675]
 [ -24077.42072308  209139.82861858   30004.74545382]]
Fn: [[ 1.46881478e-06  1.95798023e-04 -3.92600780e-02]
 [-1.94843953e-04  1.93788384e-06  7.15646018e-02]
 [ 3.74449289e-02 -7.49804314e-02  1.00000000e+00]]
F: [[-2.53306317e-01  5.03283835e-01  4.20292276e-01]
 [ 7.00311728e-01 -9.79188287e-02 -7.48715139e-02]
 [-1.60571540e-02 -2.73425826e-03 -1.66173717e-04]]
abs scale: 0.1218 | progress: 782 | len(src): 169 
85618.80659300767
En: [[-1.10345839e-04 -9.17225375e-02  1.57399977e-01]
 [ 9.10034212e-02 -9.21227320e-04  6.83303448e-01]
 [ 1.57478228e-01  6.83246410e-01  1.04380652e-03]]
E: [[ 145091.80414304 -266281.33584603   59071.86199513]
 [-362659.6878563    61191.52453214 -291689.29397634]
 [  3

19447.442646661802
En: [[ 6.80378795e-06 -1.31291803e-02  3.56372863e-01]
 [ 1.31982212e-02  6.20482364e-06  6.10593683e-01]
 [-3.57458730e-01 -6.09959361e-01  5.98805665e-05]]
E: [[-154448.21106252  275446.93848521  -64229.08005002]
 [ 209333.40176425  292442.22544164  247012.32595656]
 [ -80330.09308883  302758.75820905    4914.3094491 ]]
Fn: [[-1.95469852e-06  1.99814272e-03 -2.59563075e-01]
 [-1.99268188e-03  2.08759796e-05 -5.94847153e-01]
 [ 2.58254339e-01  5.84379951e-01  1.00000000e+00]]
F: [[-2.96663986e-01  5.29078233e-01  3.64059857e-01]
 [ 4.02087411e-01  5.61722765e-01  1.49390649e-01]
 [ 7.10404950e-05  1.54932873e-03 -6.33821747e-04]]
abs scale: 0.0556 | progress: 794 | len(src): 266 
85986.00399730714
En: [[-2.50014844e-06  7.54483097e-04 -1.51471514e-01]
 [-7.58353602e-04 -6.59744395e-06  6.90692263e-01]
 [ 1.54675009e-01 -6.89981938e-01 -2.07707015e-06]]
E: [[-125009.63713284  320197.59728813  -28712.07719758]
 [  -3725.45955305  346164.90191962   79614.79273995]
 [-1

## Analysing the performance of our algorithm:

- Our algorithm works pretty well on straight line trajectories, the prediction is very accurate and error rate is less. 
- Our algorithm is a little less efficient on curved trajectories; it is not able to detect the curves in the trajectory with complete accuracy. 
- Once a mistake is made in the curved trajectories, it getrs compounded with each new observation because of the cumulative effect of the updates. 
- Rotations are not caputured well because in the RANSAC loop, while trying to estimate a better fundamental matrix, we try to use far-apart points. However this is counter-intuitive to finding rotations in the image, because far apart points don't have much of an observable rotation between frames. 

## Other ways of computing the absolute scale
2 possible methods:
- Use a second sensor, whose data is given as part of the dataset.
- Detect known objects within the scene, via corner detection. Then use these known values to estimate distance. For example, assuming the length of a car is normally around 9 feet. Observing cars in the image, and their motion across images can give you the scaling factor.